# Dropping a ball
Written by Tim Atherton for Computational Physics 
January 16, 2020

This program aims to simulate the motion of a ball dropped from an initial height of $y0$ under gravity $g$ using the Euler discretization.

## Import libraries for plotting and numerical work

In [ ]:
import sys
print(sys.version)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Set up general parameters for the calculation
- Gravitational acceleration constant: $10 \frac{m}{s^2}$

- Initial height: $y_0$ in $m$
- Initial velocity: $v_0$ in $\frac{m}{s}$
- Initial time: $t_0$ in $s$
- Time resolution: $dt$ in $0.1s$
- Total simulation time: $t_f$ in $s$

In [ ]:
g = 10 ; y0 = 0 ; v0 = 0 ; t0 = 0 ; dt = 0.1 ; tf = 10 

## Set number of steps to take in the trajectory calculation

In [ ]:
nsteps = int(tf/dt)+1

## Set the properties of the ball under investigations
- The properties $S_b$ of the individual balls is a collection of elements pertaining to the various physical properties of the ball in the test set.
- Property list:
    - Ball mass: $m$ in grams
    - Ball diameter: $d$ in cm
    - Ball radius: $r$ in cm
    - Ball surface area: $s_a = 4\pi r^2$ in cm$^2$ 
    - Ball material: $b_{mat}:$ $\{$'plastic':0, 'rubber':1, 'foam':2$\}$
    - Porosity: $p:$ $\{ 0.0, ..., 1.0 \}$
- Red Foam Ball
    - $S_r:$ $\{$ $m:$ 0.40, $d:$, $r:$, $s_a:$, $b_{mat}:$, $p:$, $\}$
- Black Rubber Ball
    - $S_b:$ $\{$ $m:$ 24.55, $d:$, $r:$, $s_a:$, $b_{mat}:$, $p:$, $\}$
- White Plastic Ball
    - $S_w:$ $\{$ $m:$ 2.80, $d:$, $r:$, $s_a:$, $b_{mat}:$, $p:$, $\}$

In [ ]:
S = {'color': 'none', 'm': 0.0, 'd': 0.0, 'r': 0.0, 's_a': 0.0, 'b_mat': None, 'p': 0.0}

## Create arrays for time, velocity and position

In [ ]:
t = np.linspace(t0,tf,nsteps)
v = np.zeros([nsteps])
y = np.zeros([nsteps])

## Main loop, where we successively update velocity and time according to the Euler rule

In [ ]:
y[0] = y0; v[0] = v0

for i in range(1, nsteps):
    v[i] = v[i-1] + (dt**2 * g)
    y[i] = y[i-1] + v[i] * dt

## Visualize the result

In [ ]:
%matplotlib inline
plt.plot(t,y)
plt.title('Trajectory of the {} ball without drag'.format(S['color']))
plt.xlabel('Time /s')
plt.ylabel('Height /m')